In [1]:
#part 1 - bert feature base
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
import warnings

warnings.filterwarnings('ignore')


In [2]:
train_df = pd.read_csv('train.tsv', delimiter='\t', header=None)

train_set = train_df[:1]   #取其中的3000条数据作为我们的数据集
print("Train set shape:", train_set.shape)
train_set[1].value_counts()   #查看数据集中标签的分布

Train set shape: (1, 2)


1    1
Name: 1, dtype: int64

## tutorial

In [3]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
max_length_test = 20
test_sentence = '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
# 分词 
print(tokenizer.tokenize(test_sentence))
# id - token
print(tokenizer(test_sentence))
# token_id
print(tokenizer.convert_ids_to_tokens([1, 2, 3]))
# 
tokenizer.encode_plus(
    text=test_sentence,
    return_attention_mask=True,
    max_length=100,
    ad_to_max_length=True,
    add_special_tokens=True,
    padding=True,
    return_tensors='pt'
)

Keyword arguments {'ad_to_max_length': True} not recognized.


['[CLS]', 'is', 'this', 'jack', '#', '#', 'son', '#', '#', 'ville', '?', '[SEP]', 'no', 'it', 'is', 'not', '.', '[SEP]']
{'input_ids': [101, 101, 2003, 2023, 2990, 1001, 1001, 2365, 1001, 1001, 20184, 1029, 102, 2053, 2009, 2003, 2025, 1012, 102, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[unused0]', '[unused1]', '[unused2]']


{'input_ids': tensor([[  101,   101,  2003,  2023,  2990,  1001,  1001,  2365,  1001,  1001,
         20184,  1029,   102,  2053,  2009,  2003,  2025,  1012,   102,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:
input_ids = tokenizer.encode("Hello, my dog is cuting",
                     add_special_tokens=True)
tokenizer.decode(input_ids)
tokenized = tokenizer(test_sentence, add_special_tokens=True)

tokenized.convert_to_tensors()

{'input_ids': [101, 3231, 19204, 3989, 6251, 1010, 2628, 2011, 2178, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## 利用BERT进行特征抽取

In [4]:
output = tokenizer.batch_encode_plus(
    train_df[0].values,
    return_attention_mask=True,
    return_tensors='pt',
    max_length=100,
    truncation=True,
    add_special_tokens=True,
    padding=True
)
res = model(output['input_ids'], output['token_type_ids'], output['attention_mask'])